In [17]:
import nltk
from nltk.chat.util import Chat, reflections
import pandas as pd
import glob
# !pip install faiss-cpu
import re

from sentence_transformers import SentenceTransformer
import numpy as np

import faiss
# Define chatbot responses
pairs = [
    [r"(?i).*hello.*|.*hi.*|.*hey.*", ["Hello! Welcome to our hotel. How can I assist you?", "Hi there! Looking for information about our hotel?", "Hey! How may I help you with your hotel inquiries?"]],
    [r"(?i).*rooms available.*|.*room types.*|.*accommodation options.*", ["We offer single, double, deluxe, and suite rooms with modern amenities.", "Our hotel provides a variety of rooms, including standard, luxury, and family suites.", "You can choose from single, double, executive, and penthouse suites."]],
    [r"(?i).*room rates.*|.*price of rooms.*|.*cost per night.*", ["Room rates depend on the type and season. Standard rooms start from $100 per night.", "Prices vary based on availability and season. Visit our website for the latest rates.", "Our hotel offers competitive pricing. Contact us for special discounts."]],
    [r"(?i).*amenities.*|.*facilities.*|.*services provided.*", ["We offer free Wi-Fi, a swimming pool, spa, gym, and 24/7 room service.", "Our facilities include a business center, fine dining restaurant, and airport shuttle service.", "Enjoy our rooftop lounge, in-room dining, and exclusive spa treatments."]],
    [r"(?i).*check-in time.*|.*when can i check in.*", ["Check-in time starts from 2:00 PM. Early check-in is available upon request.", "You can check in from 2:00 PM onwards. Please inform us if you need early check-in.", "Standard check-in is from 2 PM. Let us know if you have special timing needs."]],
    [r"(?i).*check-out time.*|.*when to check out.*", ["Check-out time is 12:00 PM. Late check-out may be available upon request.", "You must check out by 12:00 PM. Contact reception for late check-out options.", "Check-out is at noon. Let us know if you need to extend your stay."]],
    [r"(?i).*booking.*|.*reservation.*|.*how to book.*", ["You can book a room online through our website or call our reception for reservations.", "Visit our website or contact our front desk to make a reservation.", "To book a room, go to our booking page or call our reservation hotline."]],
    [r"(?i).*cancellation policy.*|.*refund policy.*", ["Our cancellation policy allows free cancellations up to 24 hours before check-in.", "If you cancel within 24 hours of check-in, a cancellation fee may apply.", "Full refunds are available for cancellations made at least 1 day in advance."]],
    [r"(?i).*location.*|.*where is the hotel.*|.*hotel address.*", ["Our hotel is located in the city center, near major attractions.", "You can find us at 123 Main Street, Downtown.", "Visit our website for detailed location and directions."]],
    [r"(?i).*restaurant.*|.*dining options.*|.*food services.*", ["We have a multi-cuisine restaurant, a coffee shop, and in-room dining service.", "Enjoy fine dining at our hotel restaurant or grab a quick snack at our café.", "Our hotel offers buffet breakfast, à la carte dining, and 24/7 room service."]],
    [r"(?i).*airport shuttle.*|.*transportation.*|.*pick-up service.*", ["Yes, we offer airport pick-up and drop-off services. Contact us to schedule a ride.", "Our hotel provides shuttle services to and from the airport.", "You can request an airport transfer while booking your stay."]],
    [r"(?i).*parking.*|.*car parking.*|.*is parking available.*", ["Yes, we offer free parking for all guests staying at our hotel.", "We have secure parking facilities available for guests.", "Our hotel provides both valet and self-parking options."]],
    [r"(?i).*pet policy.*|.*are pets allowed.*|.*bring my pet.*", ["Yes, our hotel is pet-friendly. Please inform us in advance if you’re bringing a pet.", "Pets are allowed in designated rooms. Additional charges may apply.", "We welcome pets, but certain restrictions apply. Contact us for details."]],
    [r"(?i).*bye.*|.*goodbye.*|.*see you later.*", ["Goodbye! Have a great stay at our hotel.", "See you later! Let us know if you need more assistance.", "Alright, take care! We hope to welcome you soon."]],
    [r'(?i).*hadith.*',['Write your Querey here.']]
]

chatbot = Chat(pairs, reflections)

def chat():
    print("Hello! I am your hotel assistant. How can I help you today?")

    while True:
        user_input = input("You: ")
        response = chatbot.respond(user_input)
        print(f"Bot: {response}")
        if response in ["Goodbye! Have a great stay at our hotel.", "See you later! Let us know if you need more assistance.", "Alright, take care! We hope to welcome you soon."]:
          break
        if response in ['Write your Querey here.']:
          u_input = input('You:')
          retrieve_similar_hadiths(u_input,model,index,hadith_df_en)
        if not response:
            response = "I'm sorry, I don't know how to respond to that."





In [24]:
def retrieve_similar_hadiths(query, model, index, hadith_df, k=5):
    # Preprocess and embed the query
    query_embedding = model.encode([clean_text(query)])

    # Search the FAISS index for similar Hadiths
    distances, indices = index.search(query_embedding, k)

    # Retrieve and print the top-k Hadiths
    for i in range(k):
        print(f"Hadith {i+1}:")
        print(hadith_df['Hadith'].iloc[indices[0][i]])
        print(f"Distance: {distances[0][i]}\n")

In [19]:
index = faiss.read_index('/hadith_faiss_en.index')

In [20]:
hadith_df_en = pd.read_csv('/cleaned_hadith_data_english.csv')


In [21]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [25]:
chat()

Hello! I am your hotel assistant. How can I help you today?
You: hadith
Bot: Write your Querey here.
You:fasting in ramadan
Hadith 1:
Narrated Abu Ayyub: The Prophet (ﷺ) as saying: If anyone fasts during Ramadan, then follows it with six days in Shawwal, it will be like a perpetual fast.
Distance: 28.837886810302734

Hadith 2:
It was narrated from Abu Hurairah that the Messenger of Allah (ﷺ) said: ‘Do not anticipate Ramadan by fasting one or two days before, except for a man who has a habitual pattern of fasting, in which case let him fast.”
Distance: 30.951087951660156

Hadith 3:
Abu Ayub narrated that : the Messenger of Allah said: "Whoever fasts Ramadan, then follows it with six from Shawwal, then that is (equal in reward) to fasting everyday."
Distance: 34.41267013549805

Hadith 4:
Narrated Abdullah ibn Abbas: The Prophet (ﷺ) said: Do not fast one day or two days just before Ramadan except in the case of a man who has been in the habit or observing a fast (on that day); and do not 

In [23]:
def clean_text(text):
    # removing special characters, digits, etc.
    if isinstance(text, str):
        text = re.sub(r'[^A-Za-z\s]', '', text)
        text = text.lower()
    else:
        text = ''
    return text